# CNN - Extract Vectors

In [5]:
import numpy as np
import pandas as pd
import os

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Feature Extractor
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove Fully Connected Layer
resnet.eval()

# Image Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the input size of CNN
    transforms.ToTensor(),          # Tensor Transformation
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])     #Normalization(Regularization)
])


def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0)      # Add Batch Dimension
        with torch.no_grad():
            features = resnet(image)
        return features.squeeze().numpy()  # Return 1D Vectors
    except Exception as e:
        print(f"Fail to Image Processing: {image_path}, Error: {e}")
        return np.zeros(2048)

/opt/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
image_folder = 'train_image'
metadata_csv = 'train.csv'

df = pd.read_csv(metadata_csv)

image_features = []

for img_id in df['Id']:
    img_path = os.path.join(image_folder, f"{img_id}.jpg")
    feature_vector = extract_features(img_path)
    image_features.append(feature_vector)

print(image_features)

[array([0.7207003 , 0.33531913, 0.646761  , ..., 0.0498631 , 0.36238784,
       0.12584233], dtype=float32), array([1.035094  , 0.55400425, 0.3023484 , ..., 0.11237787, 0.9961765 ,
       0.10534582], dtype=float32), array([0.3457755 , 0.5964498 , 0.434964  , ..., 0.20739257, 0.39715844,
       0.27706006], dtype=float32), array([0.36356813, 0.41006213, 0.25220296, ..., 0.23390673, 0.34314272,
       0.28667143], dtype=float32), array([0.2666772 , 0.7753209 , 1.1866456 , ..., 0.15841167, 0.34339103,
       0.22227708], dtype=float32), array([0.58195496, 0.19205435, 0.43363002, ..., 0.5004909 , 0.24915391,
       0.24556813], dtype=float32), array([0.32113525, 1.1775435 , 0.69176185, ..., 0.05942311, 0.4286221 ,
       0.25046286], dtype=float32), array([0.40636468, 1.4903101 , 0.16881314, ..., 0.5506169 , 0.21181531,
       0.06621227], dtype=float32), array([0.16477695, 0.36228183, 0.03958105, ..., 0.59323674, 0.14485832,
       0.32807592], dtype=float32), array([0.21512648, 0.360879

In [13]:
image_features_df = pd.DataFrame(image_features)
image_features_df.columns = [f"feature_{i}" for i in range(image_features_df.shape[1])]

df_combined = pd.concat([df, image_features_df], axis=1)
df_combined.to_csv("cnn_features_combined.csv", index=False)

print("Successfully Saved")

Successfully Saved


# XGBoost, Hyperparmeter Tuning

In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = pd.read_csv("cnn_features_combined.csv")
df = df.drop(columns=["Id"], errors="ignore")

target_column = "Pawpularity"  # Target Variable
X = df.drop(columns=[target_column])  # Independent Variable (CNN Vectors + Metadata)
y = df[target_column]  # Target Variable (Pawpularity Score)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = XGBRegressor(
    n_estimators=100,  
    learning_rate=0.1,  
    max_depth=5,  
    random_state=42
)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost - Mean Squared Error (MSE): {mse_xgb:.4f}")
print(f"XGBoost - Mean Absolute Error (MAE): {mae_xgb:.4f}")
print(f"XGBoost - R² Score: {r2_xgb:.4f}")

XGBoost - Mean Squared Error (MSE): 357.7013
XGBoost - Mean Absolute Error (MAE): 13.9953
XGBoost - R² Score: 0.1908


In [3]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Set the Hyperparameters
param_dist = {
    'n_estimators': [100, 300], 
    'max_depth': [3, 5],  
    'learning_rate': [0.05, 0.1],  
    'subsample': [0.8, 1.0],  
    'colsample_bytree': [0.8, 1.0]  
}

# 2. RandomizedSearchCV
xgb_model = XGBRegressor(random_state=42, tree_method="hist", enable_categorical=False)
xgb_cv = RandomizedSearchCV(
    xgb_model, param_distributions=param_dist, 
    n_iter=10, cv=5, scoring='r2', n_jobs=2, verbose=2, random_state=42
)
xgb_cv.fit(X_train, y_train)

# 3. Print Optimal Hyperparameter
print(f"Best XGBoost Parameters: {xgb_cv.best_params_}")
print(f"Best XGBoost R² Score: {xgb_cv.best_score_}")

# 4. Evaluate Optimal Model
best_xgb = xgb_cv.best_estimator_
y_pred_best = best_xgb.predict(X_test)
mse_best = mean_squared_error(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print(f"Best Tuned XGBoost - MSE: {mse_best:.4f}")
print(f"Best Tuned XGBoost - MAE: {mae_best:.4f}")
print(f"Best Tuned XGBoost - R² Score: {r2_best:.4f}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1.0; total time=  36.7s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1.0; total time=  34.6s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1.0; total time=  33.2s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=300, subsample=1.0; total time= 1.6min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=300, subsample=1.0; total time= 1.6min
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8; total time=  20.8s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8; total time=  20.9s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8; total time=  19.4s
[CV] END colsample_

/opt/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1.0; total time=  36.5s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1.0; total time=  35.5s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=300, subsample=1.0; total time= 1.5min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=300, subsample=1.0; total time= 1.5min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=300, subsample=1.0; total time= 1.8min
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8; total time=  19.6s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=1.0; total time=  24.5s
[CV] END colsample_bytree=1.0, learning_rate=0.05, max_depth=3, n_estimators=100, subsample=1.0; total time=  20.8s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=10